# Configuración Inicial

In [1]:
import sys
sys.path.append('../src/')
from utils import configurar_ruta_datos, cargar_datos

configurar_ruta_datos('../datos/')
datos_individuos = cargar_datos('individual')

Se han cargado 35 archivos de tipo 'individual' desde la ruta '../datos/'.


# INDIVIDUO

Genero (CH04)

In [2]:
for individuo in datos_individuos:
    individuo['CH04_str'] = 'Masculino' if individuo['CH04'] == '1' else 'Femenino'

Nivel educativo (NIVEL_ED)

In [3]:
# Crear la columna 'NIVEL_ED_str' utilizando un mapeo
nivel_educativo_map = {
    '1': 'Primario incompleto',
    '2': 'Primario completo',
    '3': 'Secundario incompleto',
    '4': 'Secundario completo',
    '5': 'Superior|universitario incompleto',
    '6': 'Superior|universitario completo',
    '7': 'Sin instrucción',
    '9': 'No sabe|No responde'
}

for individuo in datos_individuos:
    individuo['NIVEL_ED_str'] = nivel_educativo_map.get(individuo['NIVEL_ED'], 'Sin información')  # Valor por defecto

Condicion Laboral

In [4]:
for individuo in datos_individuos:
    estado = individuo['ESTADO']
    cat_ocup = individuo['CAT_OCUP']
    
    if estado == '1':
        if cat_ocup in ('1', '2'):
            individuo['CONDICION_LABORAL'] = 'Ocupado autónomo'
        elif cat_ocup in ('3', '4', '9'):
            individuo['CONDICION_LABORAL'] = 'Ocupado dependiente'
    elif estado == '2':
        individuo['CONDICION_LABORAL'] = 'Desocupado'
    elif estado == '3':
        individuo['CONDICION_LABORAL'] = 'Inactivo'
    else:
        individuo['CONDICION_LABORAL'] = 'Fuera de categoría/sin información'


Nivel Universitario

In [5]:
for individuo in datos_individuos:
    if individuo['NIVEL_ED'] == '5': # Universitario incompleto
        individuo['UNIVERSITARIO'] = '0'
    elif individuo['NIVEL_ED'] == '6': # Universitario completo
        individuo['UNIVERSITARIO'] = '1'
    else: # No aplicable
        individuo['UNIVERSITARIO'] = '2'

# Almacenamiento en nuevos archivos

In [6]:
# Define las columnas que deseas guardar para individuos y hogares
from collections import defaultdict
import csv

columnas_individuos = {'CODUSU','NRO_HOGAR','COMPONENTE','ANO4', 'TRIMESTRE','REGION','AGLOMERADO','PONDERA','CH04', 'CH04_str','CH06','CH07','CH09', 'CH15', 'NIVEL_ED','NIVEL_ED_str','ESTADO','CAT_INAC', 'CONDICION_LABORAL', 'UNIVERSITARIO','PP07H', 'PP04A'}

# INDIVIDUOS
# Agrupar datos por año y trimestre
individuos_por_trimestre = defaultdict(list)
for ind in datos_individuos:
    anio = ind.get('ANO4', 'NA')
    trimestre = ind.get('TRIMESTRE', 'NA')
    if anio != 'NA' and trimestre != 'NA':
        key = f"A{anio}_T{trimestre}"
        individuos_por_trimestre[key].append(ind)

# Crear archivos por cada grupo
for key, individuos in individuos_por_trimestre.items():
    with open(f'../datos/procesado/ind_{key}.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=list(columnas_individuos))       
        writer.writeheader()
        for ind in individuos:
            limpio = {k: v for k, v in ind.items() if k in columnas_individuos}
            writer.writerow(limpio)